In [12]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
solve_season = '2024-25'
solve_gameweek = 21
load_projections_from_file = False

In [14]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now you can import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [15]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [16]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW21 Challenge: Legends - Players who have been at their club for 5+ seasons earn double points

In [17]:
import requests
# Challenge api info = 'https://fplchallenge.premierleague.com/api/elements/21/'

# Get player data from API
# Fetch data from API
response = requests.get('https://fplchallenge.premierleague.com/api/elements/21/')
player_data = response.json()

# Create a mapping of player ID to join date
join_dates = {player['id']: player['team_join_date'] for player in player_data}

# Add team_join_date column to projections_data
projections_data['team_join_date'] = projections_data['ID'].map(join_dates)

# Convert join dates to datetime where not null
projections_data['team_join_date'] = pd.to_datetime(projections_data['team_join_date'])

# Add a column to projections_data for whether the player has been at their club for 5+ seasons, by checking if date is <=May 2021
projections_data['5+_seasons'] = projections_data['team_join_date'] <= pd.Timestamp('2021-05-01')

# Double the Predicted_Points for players who have been at their club for 5+ seasons
projections_data.loc[projections_data['5+_seasons'], 'Predicted_Points'] *= 2

### Optimisation

In [18]:
# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [
    plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Define captain variables
captain = [
    plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer")
    for i in player_ids
]

# Set the objective function (the number of points scored by the team, with captain's points doubled)
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) + \
         plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)])

# Constraints

# Total number of players = 5
model += plp.lpSum(lineup) == 5

# List players by index to be EXCLUDED from the lineup
for id in ban_ids:
    model += lineup[id] == 0

for id in force_ids:
    model += lineup[id] == 1

# Exactly one captain
model += plp.lpSum(captain) == 1

# Captain must be in the lineup
for i in range(player_count):
    model += captain[i] <= lineup[i]

# Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1

# 1-2 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 2

# 1-2 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 2

# 1-2 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) >= 1
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) <= 2

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict):
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                captain_str = " (C)" if player['Captain'] else ""
                points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

def solution_to_txt(players_dict, filename="solution.txt", encoding="utf-8"):
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    captain_str = " (C)" if player['Captain'] else ""
                    points = player['Predicted_Points'] * (2 if player['Captain'] else 1)
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")
        
solution_to_txt(selected_players, "optimal_solution.txt")
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  A.Becker - Liverpool - Cost: 5.4m - Predicted Points: 8.44

Defender:
  Alexander-Arnold - Liverpool - Cost: 7.2m - Predicted Points: 9.94

Midfielder:
  M.Salah (C) - Liverpool - Cost: 13.7m - Predicted Points: 29.84
  B.Fernandes - Man Utd - Cost: 8.3m - Predicted Points: 11.38

Forward:
  Vardy - Leicester - Cost: 5.5m - Predicted Points: 9.34

Total Predicted Points: 68.94
Total Cost: 40.1m


In [19]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

projections_data.to_csv('gameweek_projections.csv', index=False)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals,team_join_date,5+_seasons
181,310,A.Becker,Liverpool,30,Goalkeeper,5.4,8.44,90,Nott'm Forest (A),0.0,0.0,0.0,0.381,0.0,2018-07-19,True
131,235,Pickford,Everton,241,Goalkeeper,5.0,7.88,90,Aston Villa (H),0.0,0.0,0.0,0.267,0.0,2017-07-01,True
179,308,Ward,Leicester,244,Goalkeeper,4.0,7.36,90,Crystal Palace (H),0.0,0.0,0.0,0.068,0.0,2018-07-20,True
306,521,Fabianski,West Ham,172,Goalkeeper,4.1,7.08,76,Fulham (H),0.0,0.0,0.0,0.134,0.0,2018-06-20,True
240,396,Dúbravka,Newcastle,194,Goalkeeper,4.2,6.78,68,Wolves (H),0.0,0.0,0.0,0.421,0.0,2018-07-01,True
207,347,Ederson M.,Man City,30,Goalkeeper,5.3,6.72,79,Brentford (A),0.0,0.0,0.0,0.066,0.0,2017-07-01,True
51,84,Travers,Bournemouth,104,Goalkeeper,4.4,6.66,90,Chelsea (A),0.0,0.0,0.0,0.104,0.0,2018-07-01,True
29,47,Martinez,Aston Villa,10,Goalkeeper,5.0,6.54,68,Everton (A),0.0,0.0,0.0,0.333,0.0,2020-09-16,True
9,15,Raya,Arsenal,200,Goalkeeper,5.6,4.04,90,Spurs (H),0.0,0.0,0.0,0.400,0.0,2024-07-04,False
102,185,Sánchez,Chelsea,200,Goalkeeper,4.8,4.00,90,Bournemouth (H),0.0,0.0,0.0,0.341,0.0,2023-08-05,False


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals,team_join_date,5+_seasons
182,311,Alexander-Arnold,Liverpool,241,Defender,7.2,9.94,87,Nott'm Forest (A),0.096,0.283,0.352,0.381,0.108,2015-07-01,True
1,3,Gabriel,Arsenal,30,Defender,6.4,9.88,90,Spurs (H),0.059,0.078,0.132,0.400,0.085,2020-09-01,True
197,335,Robertson,Liverpool,243,Defender,5.9,8.80,79,Nott'm Forest (A),0.064,0.198,0.250,0.357,0.072,2017-07-21,True
200,339,Virgil,Liverpool,152,Defender,6.4,8.22,90,Nott'm Forest (A),0.100,0.084,0.176,0.381,0.112,2018-01-01,True
12,18,Saliba,Arsenal,73,Defender,6.3,8.08,90,Spurs (H),0.083,0.055,0.134,0.400,0.093,2019-07-25,True
251,415,Schär,Newcastle,207,Defender,5.4,7.38,90,Wolves (H),0.111,0.038,0.145,0.395,0.125,2018-07-26,True
226,377,Maguire,Man Utd,241,Defender,4.9,7.32,90,Southampton (H),0.125,0.022,0.144,0.500,0.141,2019-08-05,True
114,210,Mitchell,Crystal Palace,241,Defender,4.8,6.96,90,Leicester (A),0.058,0.083,0.136,0.333,0.066,2019-12-15,True
21,32,Cash,Aston Villa,172,Defender,4.4,6.94,76,Everton (A),0.098,0.049,0.142,0.312,0.109,2020-09-03,True
70,120,Dunk,Brighton,241,Defender,4.3,6.90,90,Ipswich (A),0.076,0.039,0.112,0.312,0.086,2009-07-01,True


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals,team_join_date,5+_seasons
193,328,M.Salah,Liverpool,63,Midfielder,13.7,14.92,90,Nott'm Forest (A),0.431,0.485,0.707,0.381,0.597,2017-07-01,True
221,366,B.Fernandes,Man Utd,173,Midfielder,8.3,11.38,87,Southampton (H),0.303,0.244,0.473,0.469,0.395,2020-01-30,True
108,199,Eze,Crystal Palace,241,Midfielder,6.6,10.26,82,Leicester (A),0.256,0.086,0.320,0.312,0.314,2020-08-28,True
62,99,Mbeumo,Brentford,38,Midfielder,7.8,10.16,90,Man City (H),0.308,0.112,0.386,0.111,0.376,2019-08-05,True
220,364,Amad,Man Utd,54,Midfielder,5.4,10.10,87,Southampton (H),0.299,0.103,0.371,0.500,0.369,2021-01-07,True
208,348,Foden,Man City,241,Midfielder,9.2,9.30,83,Brentford (A),0.360,0.302,0.553,0.294,0.464,2016-12-05,True
5,9,Martinelli,Arsenal,30,Midfielder,6.8,9.26,74,Spurs (H),0.314,0.200,0.451,0.388,0.409,2019-07-15,True
205,345,De Bruyne,Man City,21,Midfielder,9.4,8.28,60,Brentford (A),0.038,0.033,0.069,0.276,0.048,2015-08-30,True
203,342,Bernardo,Man City,173,Midfielder,6.3,8.00,90,Brentford (A),0.222,0.366,0.507,0.294,0.265,2017-07-01,True
312,530,Souček,West Ham,57,Midfielder,4.9,7.96,89,Fulham (H),0.128,0.062,0.182,0.234,0.149,2020-07-28,True


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals,team_join_date,5+_seasons
177,306,Vardy,Leicester,241,Forward,5.5,9.34,85,Crystal Palace (H),0.269,0.091,0.335,0.188,0.335,2012-07-01,True
83,148,Welbeck,Brighton,241,Forward,5.5,8.36,84,Ipswich (A),0.290,0.077,0.344,0.312,0.366,2020-10-18,True
210,351,Haaland,Man City,161,Forward,14.8,6.71,90,Brentford (A),0.591,0.201,0.673,0.294,0.935,2022-07-01,False
244,401,Isak,Newcastle,206,Forward,9.3,5.65,64,Wolves (H),0.505,0.115,0.562,0.421,0.768,2022-08-26,False
317,541,Cunha,Wolves,30,Forward,7.0,5.36,90,Newcastle (A),0.208,0.000,0.000,0.094,0.253,2023-07-01,False
2,4,Havertz,Arsenal,80,Forward,7.8,5.22,87,Spurs (H),0.320,0.165,0.433,0.375,0.424,2023-06-28,False
112,207,Mateta,Crystal Palace,73,Forward,7.2,5.11,87,Leicester (A),0.323,0.072,0.372,0.333,0.421,2022-01-31,False
67,110,Wissa,Brentford,50,Forward,6.2,5.02,87,Man City (H),0.259,0.049,0.295,0.111,0.312,2021-08-10,False
269,447,Wood,Nott'm Forest,154,Forward,6.8,4.96,87,Liverpool (H),0.264,0.057,0.306,0.154,0.319,2023-07-01,False
99,180,N.Jackson,Chelsea,189,Forward,8.1,4.86,78,Bournemouth (H),0.347,0.110,0.419,0.341,0.470,2023-07-01,False
